<a href="https://colab.research.google.com/github/manudeepsinha/AI_Assistant/blob/main/AI_Assistant_Google_Cal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Before running the code, select T4 GPU as Hardware accelerator in Google Colab
#Get the credentials.json file and upload it to the working directory
pip install langchain transformers torch langchain-community

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import logging
from datetime import datetime, timedelta
from dateutil import parser
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import GPTJForCausalLM, GPT2Tokenizer, pipeline
import torch

# Google Calendar Client
class GoogleCalendarClient:
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    SERVICE_ACCOUNT_FILE = 'credentials.json'

    def __init__(self):
        self.creds = service_account.Credentials.from_service_account_file(
            self.SERVICE_ACCOUNT_FILE, scopes=self.SCOPES)
        self.service = build('calendar', 'v3', credentials=self.creds)

    def book_appointment(self, summary, start_time, end_time):
        try:
            event = {
                'summary': summary,
                'start': {
                    'dateTime': start_time.isoformat(),
                    'timeZone': 'Asia/Kolkata',
                },
                'end': {
                    'dateTime': end_time.isoformat(),
                    'timeZone': 'Asia/Kolkata',
                },
            }
            event = self.service.events().insert(calendarId='primary', body=event).execute()
            logging.info(f'Event created: {event.get("htmlLink")}')
            return f'Appointment booked: {event.get("htmlLink")}'
        except Exception as e:
            logging.error(f'Error booking appointment: {e}')
            return 'Failed to book appointment.'

# GPT-J Agent
class GPTJAgent:
    def __init__(self, calendar_client):
        self.calendar_client = calendar_client
        self.state = {}

        # Initialize the Hugging Face pipeline
        model_name = "gpt2"
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        model = GPTJForCausalLM.from_pretrained(model_name)

        hf_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            device=0  # Use the appropriate device (0 for GPU/TPU)
        )

        self.llm = HuggingFacePipeline(pipeline=hf_pipeline)

        # Define the prompt template
        self.prompt_template = PromptTemplate(
            input_variables=["input_text"],
            template="{input_text}"
        )

        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.prompt_template
        )

    def generate_response(self, prompt):
        response = self.chain.run(input_text=prompt)
        return response.strip()

    def handle_message(self, message):
        user_id = 'default'
        if user_id not in self.state:
            self.state[user_id] = {'step': 0, 'name': None, 'date': None, 'time': None}

        step = self.state[user_id]['step']
        response = ""

        if step == 0:
            if "book" or "yes" in message.lower():
                response = "Sure, I can help with that. Can you please provide me with your name and preferred date for the appointment?"
                self.state[user_id]['step'] = 1
            else:
                response = "Sorry, I didn't understand that. Do you want to book an appointment?"

        elif step == 1:
            self.state[user_id]['name'] = message
            response = "Thank you. What date works best for you?"
            self.state[user_id]['step'] = 2

        elif step == 2:
            try:
                self.state[user_id]['date'] = parser.parse(message).date()
                response = "Got it. What time on this date works best for you?"
                self.state[user_id]['step'] = 3
            except ValueError:
                response = "I didn't understand the date. Please provide a valid date (e.g., August 10th)."

        elif step == 3:
            try:
                self.state[user_id]['time'] = parser.parse(message).time()
                start_time = datetime.combine(self.state[user_id]['date'], self.state[user_id]['time'])
                end_time = start_time + timedelta(hours=1)  # Assume 1-hour appointment
                result = self.calendar_client.book_appointment(
                    f"Appointment with {self.state[user_id]['name']}",
                    start_time,
                    end_time
                )
                response = f"Great! Your appointment has been booked for {start_time.strftime('%B %dth at %I:%M %p')}. {result} Is there anything else I can help you with?"
                self.state[user_id] = {'step': 0, 'name': None, 'date': None, 'time': None}  # Reset state
            except ValueError:
                response = "I didn't understand the time. Please provide a valid time (e.g., 10 AM)."

        return response

# Appointment Flow
class AppointmentFlow:
    def __init__(self, agent):
        self.agent = agent

    def process_conversation(self):
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Assistant: Goodbye!")
                break
            response = self.agent.handle_message(user_input)
            print(f"Assistant: {response}")

# Example usage
calendar_client = GoogleCalendarClient()
agent = GPTJAgent(calendar_client)
flow = AppointmentFlow(agent)

# Start the real-time conversation
flow.process_conversation()